<center><h1>Bank Marketing Analysis with Classification Modeling</h1></center>

# Introduction

Marketing campaigns are characterized by focusing on the customer needs and their overall satisfaction. Nevertheless, there are different variables that determine whether a marketing campaign will be successful or not. There are certain variables that we need to take into consideration when making a marketing campaign.

The goal of this notebook is to use the data to develop a strong model in order to predict which people the bank should market to for their marketing campain to get people to sign up for a term deposit.

## What is a Term Deposit?

A **Term deposit** is a deposit that a bank or a financial institurion offers with a fixed rate (often better than just opening deposit account) in which your money will be returned back at a specific maturity time. For more information with regards to Term Deposits please click on this link from Investopedia: https://www.investopedia.com/terms/t/termdeposit.asp

# About the data

Unfortunately, not much about the data is given besides for the below information. More information about where this data came from can be found at this website: 

## Bank client data:<br>
1 - **age:** (numeric)<br>
2 - **job:** type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>
3 - **marital:** marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>
4 - **education:** (categorical: primary, secondary, tertiary and unknown)<br>
5 - **default:** has credit in default? (categorical: 'no','yes','unknown')<br>
6 - **housing:** has housing loan? (categorical: 'no','yes','unknown')<br>
7 - **loan:** has personal loan? (categorical: 'no','yes','unknown')<br>
8 - **balance:** Balance of the individual.

## Related with the last contact of the current campaign:
8 - **contact:** contact communication type (categorical: 'cellular','telephone') <br>
9 - **month:** last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>
10 - **day:** last contact day of the week (categorical: 'mon','tue','wed','thu','fri')<br>
11 - **duration:** last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br>
## Other attributes:<br>
12 - **campaign:** number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>
13 - **pdays:** number of days that passed by after the client was last contacted from a previous campaign (numeric; -1 means client was not previously contacted)<br>
14 - **previous:** number of contacts performed before this campaign and for this client (numeric)<br>
15 - **poutcome:** outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')<br>

Output variable (desired target):<br>
21 - **deposit** - has the client subscribed a term deposit? (binary: 'yes','no')

# Libraries

## Libraries for plotting and analysis

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib.style as style
from matplotlib import pyplot
from numpy import where
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Libraries for Modeling

In [ ]:
from collections import Counter
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

## Data Exploration

In [ ]:
data = pd.read_csv('../input/bank-marketing-dataset/bank.csv')
data.head()

In [ ]:
data.shape

In [ ]:
#check for any Null values
nulls = []
for i in data.columns:
    nulls.append(data[i].isnull().sum())
print(nulls)

In [ ]:
#check the column data types
types=[]
for i in data.columns:
    types.append(type(data[i][0]))
print(types)

# Data Cleaning: Numeric Variables

Let's start by cleaning the numeric variables and viewing the basic statistics. 

In [ ]:
data.describe()

Before moving on we will remove "duration" from our dataset because of target leakage, or the idea that the field gives information into the future. To put it in another way, this is information that we should not have yet. As noted above this field highly affects the output because if a person had a phone call duration of 0, we can assume the caller didn't pick up the phone and therefore had no chance of a sale.

In [ ]:
data.pop("duration")

Becase "day represents the day of the week we should really treat this as a catigorical variable and not a numeric. Additionally, we will combine these values into 3 buckets: beginning of the month, middle of the month, and end of the month. This will prevent us from having 31 different one-hot encoded variables and generalize the variable so that it makes sense to the business problem at hand. After transforming "day" into a catigorical variable we will one-hot encode it with the other categoriccal variables.

In [ ]:
#start = beginning of the month
#middle = middle of the month
#end = end of the month

temp_day = []
for i in data['day']:
    if i<8:
        temp_day.append("start")
    elif i>23:
        temp_day.append("end")
    else:
        temp_day.append("middle")

data['day'] = temp_day

The variable "pdays" represents the number of days that passed by after the client was last contacted from a previous campaign. We need to also note that if the client was never contacted the value of this field is -1. Becuase a value of -1 doesn't make sense in this field and won't properly capture that a client wasn't contacted I would normally create a new binary variable but "poutcome" = nonexistant captures this for us.

Below are some interesting insights we can find from the data:

In [ ]:
neg_bal = float(len(data['balance'][data['balance']<0]))/float(len(data['balance']))
print("{:.2%}".format(neg_bal)+ " percent of people or "+str(int(neg_bal*11162))+ " people have a negative balance");

In [ ]:
first_contact = float(len(data['pdays'][data['pdays']==-1]))/float(len(data['pdays']))
print("Percentage of people that is being contacted for the first time: "+"{:.2%}".format(first_contact));

In [ ]:
plt.style.use('seaborn-whitegrid')

data.hist(bins=20, figsize=(14,10), color='#E14906')
plt.show()

As we saw from the above statistics of the numeric variables there are potential outliars. Let's talk about some of the max values for some of our variables: <br>

1 - **Balance:** It's not really unheard of to have $81,000 in your checking account. For example, this would make sense from one with a high salary, intersted in saving.<br> 

2 - **pdays:** It's a bit unusual to contact a lead after 854 days (almost 2.5 years) although it is plausable. For instance, maybe after 100 days they were removed from the marketing list but a flag was raised that might indicate that the person is likly will open an account and therefore be put back on the marketing list. Although they were removed then added back, it's possible that they never stopped counting the days in between.<br>

3 - **previous:** Like the previous variables that we discussed this one is also plausable. If a client was in multiple campaigns spanning of many quarters or even years can certainly add up to over 50 contacts.<br>

4 - **Campaign:** Being contated over 60 time for one marketing campaign is a bit unheard of even for a campaign that lasts a year. Think about it, after reaching out to a person 20 don't you think they probably aren't interested? Also, most companies won't waste money like that. This will need more investigating. <br>


# Outliar Analysis

In [ ]:
data.nlargest(20,"campaign")

Based on the above data there is no indication of an error in the above data. It looks like all of the above data all have that they were previously not contacted before. I could be that possibly that the company wanted to take an aggesive approach to marketing to fresh leads. Because of this reasoning I will not take out any observations although I will make sure to keep this in mind. Although, I decided not to throw out any variables, there is certainly a strong case to be made for tossing rows or transforming the columns to adjust for the outliers.

# Data Cleaning: Catigorical Variables

In [ ]:
#populate a list with string-type columns
str_columns = []
for i in data.columns:
    if(type(data[i][0])==str):
        str_columns.append(i)

In [ ]:
print(str_columns)

## Adjusting the Binary Categorical Variables

In [ ]:
fig, axes = plt.subplots(2,2)
fig.tight_layout(h_pad=2)

bin_list = [3,4,5,10]

for i in range(1,5):
    plt.subplot(2, 2, i)
    n = bin_list[i-1]
    counts = data[str_columns[n]].value_counts()
    plt.barh(y=counts.index, width=counts)
    plt.title('Distribution of '+str_columns[n])
    plt.ylabel(str_columns[n])
    plt.xlabel('Count')

plt.subplots_adjust(left=.25, bottom=2, right=2.75, top=4)

In [ ]:
# change the character values in the above columns to 0/1 binary values
# 0="no" and 1="yes"

data['default'] = [0 if i == "no" else 1 for i in data['default']]
data['housing'] = [0 if i == "no" else 1 for i in data['housing']]
data['loan']    = [0 if i == "no" else 1 for i in data['loan']   ]
data['deposit'] = [0 if i == "no" else 1 for i in data['deposit']]

In [ ]:
#check
print(data['default'].value_counts())
print(data['housing'].value_counts())
print(data['loan'].value_counts())
print(data['deposit'].value_counts())

In [ ]:
#adjust our string list and remove the four variables above
for i in ['default','housing','loan','deposit']:
    str_columns.remove(i)
    
print(str_columns)

In [ ]:
data.head()

## Adjusting the other categorical variables

In [ ]:
print(str_columns)

Above we adjusted the binary categorical variables and turned them into numeric variables that we can work with. Now we can transform the categorical variables that contain more than one unique value using one-hot encoding. Before we transform the variables we must view the distribtion to get an idea of the behavior for each variable.

In [ ]:
fig, axes = plt.subplots(4,2)
fig.tight_layout(h_pad=.25)

j = 0
k = 0 
for i in range(1,8):
    if k==4:
        k=0
        j=1
    plt.subplot2grid((4, 2), (k,j))
    n = i-1
    counts = data[str_columns[n]].value_counts()
    plt.barh(y=counts.index, width=counts)
    plt.title('Distribution of '+str_columns[n])
    plt.ylabel(str_columns[n])
    plt.xlabel('Count')
    k=k+1
                     
plt.subplots_adjust(left=.25, bottom=.5, right=2.75, top=5)

A few notes from the above distributions:

+ It's interesting that the largest volume of calls happen in the summer months and weakest in the colder months.
+ Remember that "poutcome" reflects how they responded to a previous campaign. We should note that about 75% in this category is labeled as "unknown" which is consistant with what we found earlier.
+ based on the demographic variables, it seems like this campaign targets middle class individuals which supports the statistics from our "balance" variable.  

This information will be valuable when we review the results of our final model

In [ ]:
# let's use one-hot encodin to change the categorical variables to numericals
data = pd.get_dummies(data)
print(len(data.columns))
print(data.columns)

In [ ]:
style.use('ggplot')
sns.set_style('whitegrid')
plt.subplots(figsize = (30,30))
## Plotting heatmap. Generate a mask for the upper triangle (taken from seaborn example gallery)
mask = np.zeros_like(data.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(data.corr(), cmap=sns.diverging_palette(20, 220, n=200), annot=True, mask=mask, center = 0, );
plt.title("Heatmap of all the Features of Train data set", fontsize = 25);

Based on the above correlation table the categorical variables that got encoded with exactly 3 unique values appear to have multicolinnearity. This can probably be attributed to the fact that 2 of the encoded values will together have information on the third. For example, if there are three choices between pcoucome (success, failure, unknown) and it wasn't a success or a failure, the data will assume it was unknown. Therefore we can remove one of these encoded variables from each of the categorical varible with three unique values.

In [ ]:
#delete: poutcome_unknown, marital_single, day_end, contact_unknown

data.pop("poutcome_unknown")
data.pop("marital_single")
data.pop("day_end")
data.pop("contact_unknown")

# Modeling

Below are a few functions to streamline our modeling

In [ ]:
def Models(models, X_train, X_test, y_train, y_test, title):
    model = models
    model.fit(X_train,y_train)
    
    X, y = Definedata()
    train_matrix = pd.crosstab(y_train, model.predict(X_train), rownames=['Actual'], colnames=['Predicted'])    
    test_matrix = pd.crosstab(y_test, model.predict(X_test), rownames=['Actual'], colnames=['Predicted'])
    matrix = pd.crosstab(y, model.predict(X), rownames=['Actual'], colnames=['Predicted'])
    
    f,(ax1,ax2,ax3) = plt.subplots(1,3,sharey=True, figsize=(20, 3))
    #f = plt.figure(figsize=(20, 3))
    
    g1 = sns.heatmap(train_matrix, annot=True, fmt=".1f", cbar=False,ax=ax1)
    g1.set_title(title)
    g1.set_ylabel('Total Deposit = {}'.format(y_train.sum()), fontsize=14, rotation=90)
    g1.set_xlabel('Accuracy score for Training Dataset = {}'.format(accuracy_score(model.predict(X_train), y_train)))
    g2 = sns.heatmap(test_matrix, annot=True, fmt=".1f",cbar=False,ax=ax2)
    g2.set_title(title)
    g2.set_ylabel('Total Deposit = {}'.format(y_test.sum()), fontsize=14, rotation=90)
    g2.set_xlabel('Accuracy score for Testing Dataset = {}'.format(accuracy_score(model.predict(X_test), y_test)))
    g3 = sns.heatmap(matrix, annot=True, fmt=".1f",cbar=False,ax=ax3)
    g3.set_title(title)
    g3.set_ylabel('Total Deposit = {}'.format(y.sum()), fontsize=14, rotation=90)
    g3.set_xlabel('Accuracy score for Total Dataset = {}'.format(accuracy_score(model.predict(X), y)))
    
    plt.show()
    return y, model.predict(X)

In [ ]:
import plotly.graph_objects as go

def Featureimportances(models, X_train, y_train):
    model = models
    model.fit(X_train,y_train)
    importances = model.feature_importances_
    features = data.columns
    if len(importances)<len(features): 
        features = data.columns[:len(importances)]
    else:
        importances = model.feature_importances_[:len(features)]
    imp = pd.DataFrame({'Features': features, 'Importance': importances})
    imp = imp.sort_values(by = 'Importance', ascending=False)[:15]
    imp['Sum Importance'] = imp['Importance'].cumsum()
    
    fig = go.Figure()
    fig.add_trace(go.Bar(x=imp.Features,y=imp.Importance, marker=dict(color=list(range(20)), colorscale="Sunsetdark")))

    fig.update_layout(title="Feature Importance",
                                 xaxis_title="Features", yaxis_title="Importance",title_x=0.5, paper_bgcolor="mintcream",
                                 title_font_size=20)
    fig.show()

In [ ]:
def Definedata():
    # define dataset
    X=data.drop(columns=['deposit']).values
    y=data['deposit'].values
    return X, y

In [ ]:
# define dataset
data.dropna(inplace=True)
X=data.drop(columns=['deposit']).values
y=data['deposit'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)

In [ ]:
title = 'LogisticRegression'
%time Models(LogisticRegression(),X_train, X_test, y_train, y_test, title) 

In [ ]:
title = 'GaussianNB'
%time Models(GaussianNB(),X_train, X_test, y_train, y_test, title)

In [ ]:
title = 'KNeighborsClassifier'
%time Models(KNeighborsClassifier(n_neighbors=1),X_train, X_test, y_train, y_test, title)

In [ ]:
title = 'DecisionTreeClassifier'
%time Models(DecisionTreeClassifier(max_depth=14),X_train, X_test, y_train, y_test, title)

In [ ]:
title = 'RandomForestClassifier'
%time Models(RandomForestClassifier(),X_train, X_test, y_train, y_test, title)

In [ ]:
title = 'GradientBoostingClassifier'
%time Models(GradientBoostingClassifier(n_estimators=500, learning_rate=1, max_features=2, max_depth=2, random_state=0),X_train, X_test, y_train, y_test, title)

In [ ]:
title = 'XGBClassifier'
%time Models(XGBClassifier(),X_train, X_test, y_train, y_test, title)

In [ ]:
title = 'LGBMClassifier'
%time Models(LGBMClassifier(),X_train, X_test, y_train, y_test, title)

In [ ]:
title = 'LinearDiscriminantAnalysis'
%time Models(LinearDiscriminantAnalysis(),X_train, X_test, y_train, y_test, title)

# Analyze the Strongest Model

Let's take a closer look at the Random Forrest Model since that was one of our most predictive models

In [ ]:
from sklearn.metrics import confusion_matrix,auc,roc_curve

title = 'RandomForestClassifier'
y, ypred =  Models(RandomForestClassifier(),X_train, X_test, y_train, y_test, title)

fpr, tpr, thresholds = roc_curve(y, ypred)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',label='AUC = %0.3f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.0])
plt.ylim([-0.1,1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Above we have our 3 confustion matricies that were seen before along with an ROC curve.To briefly analyze the confusion matricies, we have a strong Accuracy for our test set with a value of about 0.71 and a strong Accuacy for our full dataset with an Accuracy of about 0.91. 

Below the confustion matricies, we can see the ROC curve with a value of 0.904 which is very strong. The ROC curve shows the trade-off between sensitivity (or TPR) and specificity (1 – FPR). Classifiers that give curves closer to the top-left corner indicate a better performance. As a baseline, a random classifier is expected to give points lying along the diagonal (FPR = TPR). The closer the curve comes to the 45-degree diagonal of the ROC space, the less accurate the test. In other words, the 45-degree line represents randomly choosing an outcome.

In [ ]:
Featureimportances(RandomForestClassifier(), X_train, y_train)

Right off the bat we can point out that our two most important features in our model are balance and age. Athough this chart doesn't tell us the direction of the feature (if it's positively or negativly affecting the outcome taking everything else constant) we can imply a few things. First, the larger the balance the more money is saved in the bank. The more money that's saved in the bank the more likely you'll have extra money to put towards a term deposti. Second, the older a person is, the more time they have had accumulating wealth. Therefore, they are more likely to have extra money to put toward a term deposit. It might be important to point out that the next three features are attributed to the campaign. 

It's also important to note that although there are two powerful features, there is a healthy amount of importance distributed among the top 10 features in our model. 

# Conclusion & Future Work

In total of 9 algorithms were used to classify whether a person would open a term deposit with the bank. Three metrics were used in evaluating their performances. Performance improvement could be achieved through developing an ensambled model, a combination of different algorithms. Although we focused on the Random Forest Model, it could be observed that the 3 following algorithms outperformed other models in terms of the Accuracy.

1. RandomForestClassifier 
2. GradientBoostingClassifier
3. XGBClassifier

It is worth noting that, understanding the business problem, how the dataset is built, and deeply understanding the variables at hand is essential for building an acccurate and significant model. 

Future work will include a comprehensive tuning of these 4 algorithms. 

# References

[1] https://www.kaggle.com/janiobachmann/bank-marketing-campaign-opening-a-term-deposit 

[2] https://www.kaggle.com/winternguyen/credit-card-100-fraud-detection

[3] https://archive.ics.uci.edu/ml/datasets/Bank+Marketing
